In [1]:
import numpy as np

# State transition model

In [2]:
def f(x, u, w, dt):
    '''
    Arguments:
        x: state vectors of each particle (= [[x, y, theta], ...]^T)
        u: control input vectors of each particle (= [[v, omega], ...]^T)
        w: noise vectors of each particle (= [[w_v, w_omega, gamma], ...]^T)
        dt: integral interval (scalar)
    Return:
        updated state vector (same size w/ x)
    '''
    
    theta = x[2, :]
    
    v = u[0, :] + w[0, :]
    omega = u[1, :] + w[1, :]
    r = v / omega
    
    gamma = w[2, :]
    
    dx = -r * np.sin(theta) + r * np.sin(theta + omega * dt)
    dy =  r * np.cos(theta) - r * np.cos(theta + omega * dt)
    dtheta = omega * dt + gamma * dt
    
    delta = np.array([
        dx,
        dy,
        dtheta
    ])
    
    x_new = x + delta
    
    return x_new